In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def generateNRiblets_v2(outfile, N, d, h, theta, l, d_frac, L_edge, R_edge, plotRiblets=False):
    
    '''generates the vertices for multiple riblets.  returns a list of 3D cartesian coords.
    this version returns all the vertices on the bottom row, followed by the middle row, then the top row.
    then repeats for the other vertices in the z-direction.
    
    This version has also been updated to include large blocks either side of the riblet arrangement in order to allow for poisseuille flow 
    to develop and also to help with the numerics at the outlflow
    
    d: riblet spacing
    h: riblet height
    theta: sharpness of riblet
    l: height of the system (distance to the lid driving the flow)
    L/R_offset: the length of the inlet/outlet blocks'''
    
    d /= 2. #more accurately represents the spacing of the riblets
    w = h*math.tan(math.radians(theta/2)) #half of the base of the riblet triangle
    z = 0.1 #the length of the system in the z-direction.  Currently just used to make a dummy dimension for OpenFOAM's 2D quirks
    d_frac = 0.4 #the fraction of the separation used as the base for the trapezoid forming a side of the prism riblet
    riblet_length = 2.*(d+w) #the total length of the riblet in the x-direction
    
    n_bottom = 6 #no. of vertices in the bottom row of a single riblet
    n_middle = 5 #no. of vertices in the middle row of a single riblet
    n_top = 5 #no. of vertices in top row of a single riblet
    
    bottom_vertices = [None]*(n_bottom*N) #make the lists slightly too big then remove the Nones later on
    middle_vertices = [None]*(n_middle*N) 
    
    #fill up the vertices repeating, then remove duplicates
    for i in range(N):
        #bottom vertices
        bottom_vertices[0+i*n_bottom] = (0 + i*riblet_length, 0, 0) #origin
        bottom_vertices[1+i*n_bottom] = ((1.-d_frac)*d + i*riblet_length, 0, 0) #RHS edge of first cuboid
        bottom_vertices[2+i*n_bottom] = (d + i*riblet_length, 0, 0) #base of riblet LHS
        bottom_vertices[3+i*n_bottom] = (d+2.*w + i*riblet_length, 0, 0) #base of riblet RHS
        bottom_vertices[4+i*n_bottom] = ((1.+d_frac)*d + 2.*w + i*riblet_length, 0, 0) #LHS of edge cuboid
        bottom_vertices[5+i*n_bottom] = (2.*(d+w) + i*riblet_length, 0, 0) #RHS of system base
        
        #middle vertices
        middle_vertices[0+i*n_middle] = (0 + i*riblet_length, h, 0)
        middle_vertices[1+i*n_middle] = ((1.-d_frac)*d + i*riblet_length, h, 0)
        middle_vertices[2+i*n_middle] = (d+w + i*riblet_length, h, 0)
        middle_vertices[3+i*n_middle] = ((1.+d_frac)*d + 2.*w + i*riblet_length, h, 0)
        middle_vertices[4+i*n_middle] = (2.*(d+w) + i*riblet_length, h, 0) 
        
        
    bottom_vertices = [np.array(v) for v in bottom_vertices if v]    
    middle_vertices = [np.array(v) for v in middle_vertices if v]
    top_vertices = [v + np.array([0, (l-h), 0]) for v in middle_vertices] #the top row of vertices at the top edge of the system
    
    all_vertices = bottom_vertices
    all_vertices.extend(middle_vertices)
    all_vertices.extend(top_vertices)
    
    
    vertices_other = [v + np.array([0., 0., z]) for v in all_vertices] #add the other vertices in the z-direction
    all_vertices.extend(vertices_other)
    
    all_vertices_LoL = [list(a) for a in all_vertices] #creates list of lists
    
    all_vertices_LoLs = [['{:.5f}'.format(x) for x  in sublist] for sublist in all_vertices_LoL] #converts the sublists into strings
    
#     for i, elem in enumerate(all_vertices_LoLs):
#         print(str(elem)+" "+str(i))
        
    #filtered_vertices = [x if x not in filtered_vertices for x in all_vertices_LoL]
    filtered_vertices = list()
    for sublist in all_vertices_LoLs:
        if sublist not in filtered_vertices:
            filtered_vertices.append(sublist)
    
    filtered_vertices = [[float(x) for x in sublist] for sublist in filtered_vertices] #converts back to floats due to legacy formatting
    
    #now we handle the edge block vertices
    edgeBlock_vertices = []
    R_edge += N*riblet_length #move the RHS edge past all the riblets
    
    L_vertices = [np.array([L_edge, 0, 0])]
    L_vertices.append(np.array([L_edge, h, 0]))
    L_vertices.append(np.array([L_edge, l, 0]))
    L_vertices_other = [v + np.array([0., 0., z]) for v in L_vertices]
    
    edgeBlock_vertices.extend(L_vertices)
    edgeBlock_vertices.extend(L_vertices_other)
    
    R_vertices = [np.array([R_edge, 0, 0])]
    R_vertices.append(np.array([R_edge, h, 0]))
    R_vertices.append(np.array([R_edge, l, 0]))
    R_vertices_other = [v + np.array([0., 0., z]) for v in R_vertices]
    
    edgeBlock_vertices.extend(R_vertices)
    edgeBlock_vertices.extend(R_vertices_other)

    if plotRiblets:
        plt.figure(figsize=(20,10))
        plt.scatter(filtered_vertices[0], filtered_vertices[1])
        plt.show()
        

    for i, elem in enumerate(filtered_vertices):
        outfile.write("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //vertex "+str(i)+"\n")
        
    outfile.write('\n')
    for i, elem in enumerate(edgeBlock_vertices):
        outfile.write("\t({0:5.3f}  {1:5.3f}  {2:5.3f})".format(elem[0], elem[1], elem[2]) + "\t //(edge block) vertex "+str(i)+"\n")

In [4]:
def generateHexBlocks(outfile, N):
    '''method to generate hex blocks for N prism riblets for use in openfoam.
    this version has been updated to include the inlet/outlet blocks
    N = no. of riblets'''
    
    #the strings below are used to have their values read in from a file in the blockMeshDict folder
    ribletX_nC = '$ribletX_nC' #the no. of x-cells in the blocks on/above the riblets
    plateauX_nC = '$plateauX_nC' #the no. of x-cells in the blocks on/above the space between riblets
    surfaceY_nC = '$surfaceY_nC' #the no. of y-cells in the blocks on the surface
    fluidY_nC = '$fluidY_nC' #the no. of y-cells in the blocks in the surrounding fluid (e.g. ocean)
    
    LBlockX_nC = '$LBlockX_nC' #the no. of x-cells in the LHS inlet block
    RBlockX_nC = '$RBlockX_nC' #the no. of x-cells in the RHS outlet block
    
    nCells_z = '1' #only one cell in z-direction
    
    n_blocks = 8 #this is the number of blocks in each riblet subsystem.  Just used for keeping track of the indexing
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system (first z-slice). -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    l0 = z0+nt #the index of the first vertex in the inlet block
    r0 = l0+6 #the index of the first index in the outlet block
    
    hex_list = [] #this is the list that will hold all the hex blocks
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along properly when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        #the lower row of blocks, along the riblet surface
        block0 = [0+bot_offset, 1+bot_offset, nb+1+mid_offset, nb+0+mid_offset, 0+bot_offset+z0, 1+bot_offset+z0, nb+1+mid_offset+z0, nb+0+mid_offset+z0]
        block1 = [1+bot_offset, 2+bot_offset, nb+2+mid_offset, nb+1+mid_offset, 1+bot_offset+z0, 2+bot_offset+z0, nb+2+mid_offset+z0, nb+1+mid_offset+z0]
        block2 = [3+bot_offset, 4+bot_offset, nb+3+mid_offset, nb+2+mid_offset, 3+bot_offset+z0, 4+bot_offset+z0, nb+3+mid_offset+z0, nb+2+mid_offset+z0]
        block3 = [4+bot_offset, 5+bot_offset, nb+4+mid_offset, nb+3+mid_offset, 4+bot_offset+z0, 5+bot_offset+z0, nb+4+mid_offset+z0, nb+3+mid_offset+z0]
        
        #the upper row of blocks, bulk fluid environment
        block4 = [nb+0+mid_offset, nb+1+mid_offset, nm+1+top_offset, nm+0+top_offset, nb+0+mid_offset+z0, nb+1+mid_offset+z0, nm+1+top_offset+z0, nm+0+top_offset+z0]
        block5 = [nb+1+mid_offset, nb+2+mid_offset, nm+2+top_offset, nm+1+top_offset, nb+1+mid_offset+z0, nb+2+mid_offset+z0, nm+2+top_offset+z0, nm+1+top_offset+z0]
        block6 = [nb+2+mid_offset, nb+3+mid_offset, nm+3+top_offset, nm+2+top_offset, nb+2+mid_offset+z0, nb+3+mid_offset+z0, nm+3+top_offset+z0, nm+2+top_offset+z0]
        block7 = [nb+3+mid_offset, nb+4+mid_offset, nm+4+top_offset, nm+3+top_offset, nb+3+mid_offset+z0, nb+4+mid_offset+z0, nm+4+top_offset+z0, nm+3+top_offset+z0]
        
        hex_list.append(block0)
        hex_list.append(block1)
        hex_list.append(block2)
        hex_list.append(block3)
        hex_list.append(block4)
        hex_list.append(block5)
        hex_list.append(block6)
        hex_list.append(block7)
        
    
    #now we print the blocks in the correct format so they can be pasted into the blockMeshDict file.
    block_i = 0
    riblet_i = 0
    for i, elem in enumerate(hex_list):
        
        block_i = i%(n_blocks)
        
        elem = [str(v) for v in elem]
        vertex_string = " ".join(elem) #the list of the vertices in the block

        nCells_x = ""
        nCells_y = ""
        xGrading = "1"
        yGrading = "$b_grading_y"
        zGrading = "$z_grading"
        
        if block_i%(n_blocks//2) == 0 or block_i%(n_blocks//2) == 3:
            nCells_x = plateauX_nC
        else:
            nCells_x = ribletX_nC
            
        if i%n_blocks < 4:
            nCells_y = surfaceY_nC
        else:
            nCells_y = fluidY_nC
            yGrading = "$t_grading_y"
            
        if i%4==1:
            xGrading = "$l_grading_x"
        if i%4==2:
            xGrading = "$r_grading_x"
            
        #if i%8 > 3:
            
            
        hex_string = "\thex ("+vertex_string+") \t("+nCells_x+" "+nCells_y+" "+nCells_z+") \tsimpleGrading ("+xGrading+" "+yGrading+" "+zGrading+") "+("\t//riblet "+str(riblet_i)+", block "+str(block_i)).rjust(25, " ")
        outfile.write(hex_string+"\n")
        if i%n_blocks == n_blocks-1 and i > 0:
            riblet_i += 1
            outfile.write('\n')
            
            
    #now we handle the inlet/outlet blocks
    edge_hex_list = []
    #LHS
    L_block0 = [l0, 0, nb, l0+1,         3+l0, z0+0, z0+nb, 3+l0+1]
    L_block1 = [l0+1, nb, nm, l0+2,      3+l0+1, z0+nb, z0+nm, 3+l0+2]
    
    #RHS
    R_block0 = [nb-1, r0, r0+1, nm-1,    z0+nb-1, 3+r0, 3+r0+1, z0+nm-1]
    R_block1 = [nm-1, r0+1, r0+2, nt-1,  z0+nm-1, 3+r0+1, 3+r0+2, z0+nt-1]
    
    edge_hex_list.append(L_block0)
    edge_hex_list.append(L_block1)
    edge_hex_list.append(R_block0)
    edge_hex_list.append(R_block1)
    
    for i, elem in enumerate(edge_hex_list):
        
        elem = [str(v) for v in elem]
        vertex_string = " ".join(elem)
        
        sideID = "LHS"
        blockID = "0"
        nCells_x = LBlockX_nC
        nCells_y = surfaceY_nC
        
        xGrading = "$inlet_xGrading"
        yGrading = "$b_grading_y"
        
        if i >= 2:
            sideID = "RHS"
            nCells_x = RBlockX_nC
            xGrading = "$outlet_xGrading"
            
        if i%2:
            blockID = "1"
            nCells_y = fluidY_nC
            yGrading = "$t_grading_y"
            
            
        hex_string = "\thex ("+vertex_string+") \t("+nCells_x+" "+nCells_y+" "+nCells_z+") \tsimpleGrading ("+xGrading+" "+yGrading+" 1) "+("\t//"+str(sideID)+", block "+str(blockID)).rjust(22, " ")
        outfile.write(hex_string+"\n")

#generateHexBlocks(2) 

In [5]:
def generateFloorFaces(outfile, N):
    '''generates the faces for the bottom floor of the system.
    these are defined anticlockwise to the face normal vector pointing out of the system
    this has been updated to include the inlet/outlet blocks
    N: no. of riblets'''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    l0 = z0+nt #the index of the first vertex in the inlet block
    r0 = l0+6 #the index of the first index in the outlet block
    
    face_list = []
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along properly when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        face0 = (0+bot_offset, z0+0+bot_offset, z0+1+bot_offset, 1+bot_offset)
        face1 = (1+bot_offset, z0+1+bot_offset, z0+2+bot_offset, 2+bot_offset)
        face2 = (2+bot_offset, z0+2+bot_offset, z0+2+nb+mid_offset, 2+nb+mid_offset)
        face3 = (2+nb+mid_offset, z0+2+nb+mid_offset, z0+3+bot_offset, 3+bot_offset)
        face4 = (3+bot_offset, z0+3+bot_offset, z0+4+bot_offset, 4+bot_offset)
        face5 = (4+bot_offset, z0+4+bot_offset, z0+5+bot_offset, 5+bot_offset)
        
        face_list.append(face0)
        face_list.append(face1)
        face_list.append(face2)
        face_list.append(face3)
        face_list.append(face4)
        face_list.append(face5)
        
    #the edge blocks
    #LHS
    face_list.append((l0, 3+l0, z0+0, 0))
    #RHS
    face_list.append((nb-1, z0+nb-1, r0+3, r0))
        
    outfile.write('\tlowerWall\n')
    outfile.write('\t{\n')
    outfile.write('\t\ttype wall;\n')
    outfile.write('\t\tfaces\n')
    outfile.write('\t\t(\n')
    
    for i in face_list:
        i = [str(f) for f in i]
        outfile.write('\t\t('+' '.join(i)+')\n')
    
    outfile.write('\t\t);\n')
    outfile.write('\t}\n')

In [6]:
def generateTopFaces(outfile, N):
    '''generates the faces for the top wall of the system.
    these are defined anticlockwise wrt the normal face vector pointing out of the system.
    updated to include the inlet/outlet blocks
    N: no. of riblets'''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    l0 = z0+nt #the index of the first vertex in the inlet block
    r0 = l0+6 #the index of the first index in the outlet block
    
    face_list = []
    
    for i in range(nt-nm-1):
        face = (nm+i, nm+i+1, z0+nm+i+1, z0+nm+i)
        face_list.append(face)
        
    #the edge blocks
    #LHS
    face_list.append((l0+2, nm, z0+nm, 3+l0+2))
    #RHS
    face_list.append((nt-1, r0+2, 3+r0+2, z0+nt-1))
        
    outfile.write('\tupperWall\n')
    outfile.write('\t{\n')
    outfile.write('\t\ttype patch;\n')
    outfile.write('\t\tfaces\n')
    outfile.write('\t\t(\n')
    
    for i in face_list:
        i = [str(f) for f in i]
        outfile.write('\t\t('+' '.join(i)+')\n')
    
    outfile.write('\t\t);\n')
    outfile.write('\t}\n')

In [7]:
def generateSideFaces(outfile, N):
    '''generates the faces for the LHS and RHS walls in the system.
    these are defined anticlockwise wrt the normal face vector pointing out of the system.
    this has been updated to include the inlet/outlet blocks.  If you need the internal faces at the edge of the riblet system,
    they'll be in the old version of the blockMesh generator
    N: no. of riblets
    '''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
        
    l0 = z0+nt #the index of the first vertex in the inlet block
    r0 = l0+6 #the index of the first index in the outlet block
    
    face_list_LHS = []
    face_list_RHS = []
    
    face0_LHS = (l0, l0+1, 3+l0+1, 3+l0)
    face1_LHS = (l0+1, l0+2, 3+l0+2, 3+l0+1)
    face_list_LHS.append(face0_LHS)
    face_list_LHS.append(face1_LHS)
    
    face0_RHS = (r0, 3+r0, 3+r0+1, r0+1)
    face1_RHS = (r0+1, 3+r0+1, 3+r0+2, r0+2)
    face_list_RHS.append(face0_RHS)
    face_list_RHS.append(face1_RHS)
    
    outfile.write('\tleftWall\n')
    outfile.write('\t{\n')
    outfile.write('\t\ttype patch;\n')
    outfile.write('\t\tfaces\n')
    outfile.write('\t\t(\n')
    
    for i in face_list_LHS:
        i = [str(f) for f in i]
        outfile.write('\t\t('+' '.join(i)+')\n')
    
    outfile.write('\t\t);\n')
    outfile.write('\t}\n')
    
    outfile.write('\n\n')
    
    outfile.write('\trightWall\n')
    outfile.write('\t{\n')
    outfile.write('\t\ttype patch;\n')
    outfile.write('\t\tfaces\n')
    outfile.write('\t\t(\n')
    
    for i in face_list_RHS:
        i = [str(f) for f in i]
        outfile.write('\t\t('+' '.join(i)+')\n')
    
    outfile.write('\t\t);\n')
    outfile.write('\t}\n')
    

In [29]:
def generateFrontFaces(outfile, N):
    
    '''generates the faces for the front wall (technically the inlet, kinda) of the system.
    these are defined anticlockwise wrt the normal face vector pointing out of the system.
   
    N: no. of riblets
    '''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    face_list_front = []
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        face0 = (0+bot_offset, 1+bot_offset, nb+1+mid_offset, nb+mid_offset)
        face1 = (1+bot_offset, 2+bot_offset, nb+2+mid_offset, nb+1+mid_offset)
        face2 = (3+bot_offset, 4+bot_offset, nb+3+mid_offset, nb+2+mid_offset)
        face3 = (4+bot_offset, 5+bot_offset, nb+4+mid_offset, nb+3+mid_offset)
        
        face4 = (nb+0+mid_offset, nb+1+mid_offset, nm+1+top_offset, nm+0+top_offset)
        face5 = (nb+1+mid_offset, nb+2+mid_offset, nm+2+top_offset, nm+1+top_offset)
        face6 = (nb+2+mid_offset, nb+3+mid_offset, nm+3+top_offset, nm+2+top_offset)
        face7 = (nb+3+mid_offset, nb+4+mid_offset, nm+4+top_offset, nm+3+top_offset)
        
        face_list_front.append(face0)
        face_list_front.append(face1)
        face_list_front.append(face2)
        face_list_front.append(face3)
        face_list_front.append(face4)
        face_list_front.append(face5)
        face_list_front.append(face6)
        face_list_front.append(face7)
        
            outfile.write('\tfrontWall\n')
            outfile.write('\t{\n')
            outfile.write('\t\ttype patch;\n')
            outfile.write('\t\tfaces\n')
            outfile.write('\t\t(\n')

            for i in face_list_front:
                i = [str(f) for f in i]
                outfile.write('\t\t('+' '.join(i)+')\n')

            outfile.write('\t\t);\n')
            outfile.write('\t}\n')

        

In [30]:
def generateBackFaces(outfile, N):
    
    '''generates the faces for the back wall (technically the outlet, kinda) of the system.
    these are defined anticlockwise wrt the normal face vector pointing out of the system.
    
    as these are on the opposite side of the system, the vertices are "rotated" in the opposite direction to the 
    front faces.
   
    N: no. of riblets
    '''
    
    nb = 6*N - (N-1) #the no. of vertices on the bottom row. -1 for the index of the last bottom vertex. nb is the index of the first mid vertex
    nm = 2*nb - N #the no of vertices in the middle and bottom rows. -1 for the index of the last middle vertex. nm is the index for the first top vertex
    nt = 3*nb - 2*N #the total number of vertices in the system. -1 for the very last index.  The other z-slice starts at index nt
    z0 = nt #the index of the first vertex in the other z slice
    
    face_list_back = []
    
    for i in range(N):
        bot_offset = 5*i #this moves everything along when we move to the next riblet
        mid_offset = 4*i
        top_offset = 4*i
        
        face0 = (z0+bot_offset+1, z0+bot_offset+0, z0+nb+0+mid_offset, z0+nb+1+mid_offset)
        face1 = (z0+bot_offset+2, z0+bot_offset+1, z0+nb+1+mid_offset, z0+nb+2+mid_offset)
        face2 = (z0+bot_offset+4, z0+bot_offset+3, z0+nb+2+mid_offset, z0+nb+3+mid_offset)
        face3 = (z0+bot_offset+5, z0+bot_offset+4, z0+nb+3+mid_offset, z0+nb+4+mid_offset)
        
        face4 = (z0+nb+1+mid_offset, z0+nb+0+mid_offset, z0+nm+0+top_offset, z0+nm+1+top_offset)
        face5 = (z0+nb+2+mid_offset, z0+nb+1+mid_offset, z0+nm+1+top_offset, z0+nm+2+top_offset)
        face6 = (z0+nb+3+mid_offset, z0+nb+2+mid_offset, z0+nm+2+top_offset, z0+nm+3+top_offset)
        face7 = (z0+nb+4+mid_offset, z0+nb+3+mid_offset, z0+nm+3+top_offset, z0+nm+4+top_offset)
        
        
        face_list_back.append(face0)
        face_list_back.append(face1)
        face_list_back.append(face2)
        face_list_back.append(face3)
        face_list_back.append(face4)
        face_list_back.append(face5)
        face_list_back.append(face6)
        face_list_back.append(face7)
        
    outfile.write('\tbackWall\n')
    outfile.write('\t{\n')
    outfile.write('\t\ttype patch;\n')
    outfile.write('\t\tfaces\n')
    outfile.write('\t\t(\n')
    
    for i in face_list_back:
        i = [str(f) for f in i]
        outfile.write('\t\t('+' '.join(i)+')\n')
    
    outfile.write('\t\t);\n')
    outfile.write('\t}\n')


In [31]:
def writeBlockmeshDict(filename, N, d, h, theta, l=200., d_frac=0.4, L_edge=-500., R_edge=500.):
    
    with open(filename, 'w+') as outfile:
        
        outfile.write('vertices\n')
        outfile.write('(\n')
        generateNRiblets_v2(outfile, N=N, d=d, h=h, theta=theta, l=H, d_frac=d_frac, L_edge=L_edge, R_edge=R_edge)
        outfile.write('\n);')
        outfile.write('\n\n\n')
    
        outfile.write('blocks\n')
        outfile.write('(\n')
        generateHexBlocks(outfile, N)
        outfile.write('\n);')
        outfile.write('\n\n\n')
        
        outfile.write('edges')
        outfile.write('(')
        outfile.write(');')
        outfile.write('\n\n\n')
        
        outfile.write('boundary\n')
        outfile.write('(\n')
        generateSideFaces(outfile, N)
        outfile.write('\n')
        generateFloorFaces(outfile, N)
        outfile.write('\n')
        generateTopFaces(outfile, N)
        outfile.write('\n')
        generateFrontFaces(outfile, N)
        outfile.write('\n')
        generateBackFaces(outfile, N)
        outfile.write(');')
        outfile.write('\n\n\n')
        
        outfile.write('mergePatchPairs')
        outfile.write('(')
        outfile.write(');')
        outfile.write('\n\n')

In [33]:
N = 3
d = 23.
h = 42.
theta=42
H = 200. #height of the computational domain

writeBlockmeshDict(filename='prismBlockMesh3D.txt', N=N, d=d, h=h, theta=theta, l=H, d_frac=0.4, L_edge=-100., R_edge=100.)